In [1]:
import pandas as pd
import numpy as np
import altair as alt
import json

# anomaly_label.csv

In [2]:
csv = pd.read_csv("anomaly_label.csv")
csv.head(5)

,BlockId,Label
0,blk_-1608999687919862906,Normal
1,blk_7503483334202473044,Normal
2,blk_-3544583377289625738,Anomaly
3,blk_-9073992586687739851,Normal
4,blk_7854771516489510256,Normal


In [4]:
len(csv)

575061

# HDFS_sample.log

In [41]:
log_data  = open('HDFS_sample.log', 'r')

lists = []
data = []

for line in log_data:
    data = line.split(' ')
    col0 = data[0]
    col1 = data[1]
    col2 = data[2]      
    col3 = data[3]
    col4 = data[4]
    col5 = data[5]
    col6 = data[6] 
    col7 = data[7:]
    lists.append([col0, col1, col2, col3, col4, col5, col6, col7])
    
raw_data = pd.DataFrame(lists, columns=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])
data = pd.concat([raw_data,csv],join="outer",axis=1) ### Combine two files

data.head(20)

,a,b,c,d,e,f,g,h,BlockId,Label
0,081109,203615,148,INFO,dfs.DataNode$PacketResponder:,PacketResponder,1,"[for, block, blk_38865049064139660, terminatin...",blk_-1608999687919862906,Normal
1,081109,203807,222,INFO,dfs.DataNode$PacketResponder:,PacketResponder,0,"[for, block, blk_-6952295868487656571, termina...",blk_7503483334202473044,Normal
2,081109,204005,35,INFO,dfs.FSNamesystem:,BLOCK*,NameSystem.addStoredBlock:,"[blockMap, updated:, 10.251.73.220:50010, is, ...",blk_-3544583377289625738,Anomaly
3,081109,204015,308,INFO,dfs.DataNode$PacketResponder:,PacketResponder,2,"[for, block, blk_8229193803249955061, terminat...",blk_-9073992586687739851,Normal
4,081109,204106,329,INFO,dfs.DataNode$PacketResponder:,PacketResponder,2,"[for, block, blk_-6670958622368987959, termina...",blk_7854771516489510256,Normal
5,081109,204132,26,INFO,dfs.FSNamesystem:,BLOCK*,NameSystem.addStoredBlock:,"[blockMap, updated:, 10.251.43.115:50010, is, ...",blk_1717858812220360316,Normal
6,081109,204324,34,INFO,dfs.FSNamesystem:,BLOCK*,NameSystem.addStoredBlock:,"[blockMap, updated:, 10.251.203.80:50010, is, ...",blk_-2519617320378473615,Normal
7,081109,204453,34,INFO,dfs.FSNamesystem:,BLOCK*,NameSystem.addStoredBlock:,"[blockMap, updated:, 10.250.11.85:50010, is, a...",blk_7063315473424667801,Normal
8,081109,204525,512,INFO,dfs.DataNode$PacketResponder:,PacketResponder,2,"[for, block, blk_572492839287299681, terminati...",blk_8586544123689943463,Normal
9,081109,204655,556,INFO,dfs.DataNode$PacketResponder:,Received,block,"[blk_3587508140051953248, of, size, 67108864, ...",blk_2765344736980045501,Normal


## Seperated into different dataframes by the values in `e`

In [42]:
g = pd.unique(data.iloc[:,4].values.ravel())
len(g)

7

In [43]:
df0 = data[data['e'] == g[0]]
df1 = data[data['e'] == g[1]]
df2 = data[data['e'] == g[2]]
df3 = data[data['e'] == g[3]]
df4 = data[data['e'] == g[4]]
df5 = data[data['e'] == g[5]]

### df1 - `dfs.DataNode$PacketResponder`

In [49]:
df = pd.DataFrame(df0.h.tolist(), index= df0.index)
t1 = pd.concat([df0, df.reindex(df0.index)], axis=1)
t1["Packet"] = t1["f"] + t1["g"]
t1["Status"] = ""
t1["Size"] = ""

t1[0] = np.where(t1[0] == 'for', t1[2], t1[0])
t1['Status'] = np.where(t1[3] == 'terminating\n', 'Terminating', t1['Status'])
t1['Size'] = np.where(t1[3] != 'terminating\n', t1[3], t1['Size'])

t1 = t1.drop(columns=['f', 'g', 'h', 1, 2, 3, 4])
t1.columns = ['c1', 'c2', 'c3', 'c4', 'c5', 'Block_ID','Label', 'Source', 'c6' ,'Packet', 'Status', 'Size'] 
t1 = t1[['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'Block_ID','Label', 'Size', 'Source','Packet', 'Status'] ]

t1.head()

,c1,c2,c3,c4,c5,c6,Block_ID,Label,Size,Source,Packet,Status
0,081109,203615,148,INFO,dfs.DataNode$PacketResponder:,None,blk_-1608999687919862906,Normal,,blk_38865049064139660,PacketResponder1,Terminating
1,081109,203807,222,INFO,dfs.DataNode$PacketResponder:,None,blk_7503483334202473044,Normal,,blk_-6952295868487656571,PacketResponder0,Terminating
3,081109,204015,308,INFO,dfs.DataNode$PacketResponder:,None,blk_-9073992586687739851,Normal,,blk_8229193803249955061,PacketResponder2,Terminating
4,081109,204106,329,INFO,dfs.DataNode$PacketResponder:,None,blk_7854771516489510256,Normal,,blk_-6670958622368987959,PacketResponder2,Terminating
8,081109,204525,512,INFO,dfs.DataNode$PacketResponder:,None,blk_8586544123689943463,Normal,,blk_572492839287299681,PacketResponder2,Terminating


In [85]:
alt.Chart(t1).mark_point().encode(
    x = 'Block_ID',
    y = 'c1',
    color='Label')

### Use decision tree to predict the Label
### Remove uni value columns, such as c4, c5
### Then calculate the accuracy rate

alt.Chart(...)

### df2 - `dfs.FSNamesystem`

In [60]:
df = pd.DataFrame(df1.h.tolist(), index= df1.index)
t2 = pd.concat([df1, df.reindex(df1.index)], axis=1)
t2["Block_ID"] = ""
t2["Size"] = ""
t2["Updated"] = ""

t2["Size"] = np.where(t2[8].str.len() < 15 , t2[8], t2["Size"])
t2["Updated"] = np.where(t2[2].str.contains("10.") , t2[2], t2["Updated"])
for i in range (0,104):
    t2["Block_ID"] = np.where(t2[i].str.contains("blk") , t2[i], t2["Block_ID"])
    
t2 = t2.drop(t2.columns[-107:-3], axis=1)
t2.columns = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7','c8', 'BlockId','Label','Block_ID', 'Size', 'Updated'] 

t2.head()

,c1,c2,c3,c4,c5,c6,c7,c8,BlockId,Label,Block_ID,Size,Updated
2,081109,204005,35,INFO,dfs.FSNamesystem:,BLOCK*,NameSystem.addStoredBlock:,"[blockMap, updated:, 10.251.73.220:50010, is, ...",blk_-3544583377289625738,Anomaly,blk_7128370237687728475,67108864\n,10.251.73.220:50010
5,081109,204132,26,INFO,dfs.FSNamesystem:,BLOCK*,NameSystem.addStoredBlock:,"[blockMap, updated:, 10.251.43.115:50010, is, ...",blk_1717858812220360316,Normal,blk_3050920587428079149,67108864\n,10.251.43.115:50010
6,081109,204324,34,INFO,dfs.FSNamesystem:,BLOCK*,NameSystem.addStoredBlock:,"[blockMap, updated:, 10.251.203.80:50010, is, ...",blk_-2519617320378473615,Normal,blk_7888946331804732825,67108864\n,10.251.203.80:50010
7,081109,204453,34,INFO,dfs.FSNamesystem:,BLOCK*,NameSystem.addStoredBlock:,"[blockMap, updated:, 10.250.11.85:50010, is, a...",blk_7063315473424667801,Normal,blk_2377150260128098806,67108864\n,10.250.11.85:50010
13,081109,204908,31,INFO,dfs.FSNamesystem:,BLOCK*,NameSystem.addStoredBlock:,"[blockMap, updated:, 10.251.110.8:50010, is, a...",blk_3640100967125688321,Normal,blk_8015913224713045110,67108864\n,10.251.110.8:50010


In [65]:
alt.Chart(t2).mark_point().encode(
    x = 'c2',
    y = 'c3',
    color='Label')

alt.Chart(...)

### df3 - `dfs.DataNode$DataXceiver`

In [8]:
df = pd.DataFrame(df2.h.tolist(), index= df2.index)
t3 = pd.concat([df2, df.reindex(df2.index)], axis=1)
t3["Block_ID"] = ""
t3["Src"] = ""
t3["Dest"] = ""
t3["Size"] = ""

t3["Size"] = np.where(t3[7].str.contains("67108864") , 67108864, t3["Size"]) 
for i in range (0,8):
    t3["Block_ID"] = np.where(t3[i].str.contains("blk") , t3[i], t3["Block_ID"])
for i in ['f', 2]:
    t3["Src"] = np.where(t3[i].str.contains("10.") , t3[i], t3["Src"])   
for i in range (3,8):
    t3["Dest"] = np.where(t3[i].str.contains("10.") , t3[i], t3["Dest"])   

t3 = t3.drop(columns=['f', 'g', 'h', 0, 1, 2, 3, 4, 5, 6, 7])
t3.columns = ['c1', 'c2', 'c3', 'c4', 'c5', 'Block_ID', 'Src', 'Dest', 'Size'] 

t3.head()

,c1,c2,c3,c4,c5,Block_ID,Src,Dest,Size
11,081109,204815,653,INFO,dfs.DataNode$DataXceiver:,blk_5792489080791696128,/10.251.30.6:33145,/10.251.30.6:50010\n,
12,081109,204842,663,INFO,dfs.DataNode$DataXceiver:,blk_1724757848743533110,/10.251.111.130:49851,/10.251.111.130:50010\n,
14,081109,204925,673,INFO,dfs.DataNode$DataXceiver:,blk_-5623176793330377570,/10.251.75.228:53725,/10.251.75.228:50010\n,
25,081109,205749,997,INFO,dfs.DataNode$DataXceiver:,blk_-28342503914935090,/10.251.123.132:57542,/10.251.123.132:50010\n,
30,081109,210037,1084,INFO,dfs.DataNode$DataXceiver:,blk_-5009020203888190378,/10.251.199.19:52622,/10.251.199.19:50010\n,


### df4 - `dfs.DataBlockScanner`

In [14]:
df = pd.DataFrame(df3.h.tolist(), index= df3.index)
t4 = pd.concat([df3, df.reindex(df3.index)], axis=1)
t4["Status"] = t4["f"] +' '+ t4["g"]

t4 = t4.drop(columns=['f', 'g','h', 0])
t4.columns = ['c1', 'c2', 'c3', 'c4', 'c5', 'BlockId', 'Status'] 

t4.head()

,c1,c2,c3,c4,c5,BlockId,Status
28,081109,205931,13,INFO,dfs.DataBlockScanner:,blk_-4980916519894289629\n,Verification succeeded
69,081109,213436,13,INFO,dfs.DataBlockScanner:,blk_-2827716238972737794\n,Verification succeeded
175,081110,002337,13,INFO,dfs.DataBlockScanner:,blk_-1547954353065580372\n,Verification succeeded
196,081110,011237,13,INFO,dfs.DataBlockScanner:,blk_6996194389878584395\n,Verification succeeded
345,081110,083453,13,INFO,dfs.DataBlockScanner:,blk_3141363517520802396\n,Verification succeeded


### df5 - `dfs.FSDataset`

In [10]:
df = pd.DataFrame(df4.h.tolist(), index= df4.index)
t5 = pd.concat([df4, df.reindex(df4.index)], axis=1)

t5 = t5.drop(columns=['f', 'g','h', 1])
t5.columns = ['c1', 'c2', 'c3', 'c4', 'c5', 'Deleting_Block(ID)', 'File'] 

t5.head()

,c1,c2,c3,c4,c5,Deleting_Block(ID),File
72,081109,213837,19,INFO,dfs.FSDataset:,blk_1781953582842324563,/mnt/hadoop/dfs/data/current/subdir5/blk_17819...
425,081110,103201,19,INFO,dfs.FSDataset:,blk_8483848473254499625,/mnt/hadoop/dfs/data/current/subdir51/blk_8483...
427,081110,103257,19,INFO,dfs.FSDataset:,blk_-8898274302731129139,/mnt/hadoop/dfs/data/current/subdir18/blk_-889...
428,081110,103320,18,INFO,dfs.FSDataset:,blk_-8014701913801168461,/mnt/hadoop/dfs/data/current/subdir27/blk_-801...
429,081110,103321,19,INFO,dfs.FSDataset:,blk_-8775602795571523802,/mnt/hadoop/dfs/data/current/subdir29/blk_-877...


### df6 - `dfs.DataNode`

In [11]:
df = pd.DataFrame(df5.h.tolist(), index= df5.index)
t6 = pd.concat([df5, df.reindex(df5.index)], axis=1)

t6 = t6.drop(columns=['g', 'h', 0, 1, 2, 3, 5])
t6.columns = ['c1', 'c2', 'c3', 'c4', 'c5', 'Src', 'Transfer_Block(ID)', 'Dest']
t6 = t6[['c1', 'c2', 'c3', 'c4', 'c5', 'Transfer_Block(ID)', 'Src', 'Dest']]

t6.head()

,c1,c2,c3,c4,c5,Transfer_Block(ID),Src,Dest
911,081110,211541,18,INFO,dfs.DataNode:,blk_4292382298896622412,10.250.15.198:50010,10.250.15.240:50010\n


## Join normal/abnormal label with HDFS sample 

,a,b,c,d,e,f,g,h,BlockId,Label
0,081109,203615,148,INFO,dfs.DataNode$PacketResponder:,PacketResponder,1,"[for, block, blk_38865049064139660, terminatin...",blk_-1608999687919862906,Normal
1,081109,203807,222,INFO,dfs.DataNode$PacketResponder:,PacketResponder,0,"[for, block, blk_-6952295868487656571, termina...",blk_7503483334202473044,Normal
2,081109,204005,35,INFO,dfs.FSNamesystem:,BLOCK*,NameSystem.addStoredBlock:,"[blockMap, updated:, 10.251.73.220:50010, is, ...",blk_-3544583377289625738,Anomaly
3,081109,204015,308,INFO,dfs.DataNode$PacketResponder:,PacketResponder,2,"[for, block, blk_8229193803249955061, terminat...",blk_-9073992586687739851,Normal
4,081109,204106,329,INFO,dfs.DataNode$PacketResponder:,PacketResponder,2,"[for, block, blk_-6670958622368987959, termina...",blk_7854771516489510256,Normal
...,...,...,...,...,...,...,...,...,...,...
575056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blk_1019720114020043203,Normal
575057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blk_-2683116845478050414,Normal
575058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blk_5595059397348477632,Normal
575059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blk_1513937873877967730,Normal
